# Парсинг с использованием selenium. Otello
Ссылка на сайт: otello.ru

In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
from selenium.webdriver.common.action_chains import ActionChains


На сайте otello.ru мы нашли следующую особенность: хотя парсинг отдельных страниц отелей очень простой и понятный, нахождение ссылок на эти страниц является неочевидной задачей - нельзя напрямую через просмотр html-кода страницы получить все ссылки. 

Мы уже нашли решение это проблемы - использование API-сервиса для нахождения внутренних ссылок. Попробуем рассмотреть другой способ - использование библиотеки selenium. selenium позволяет загружать страницы, которые используют JavaScript для динамической подгрузки контента

Напишем простой скрипт, который просто открывает главную страницу с отелями в Москве и выгружает все возможные файлы 

In [9]:

chrome_options = Options()
chrome_options.add_argument("--headless")  
service = Service('chromedriver.exe')

driver = webdriver.Chrome(service=service, options=chrome_options)

try:
     
    driver.get("https://otello.ru/hotels/moskva")
    time.sleep(5) 
    html_content = driver.page_source

    with open("otello_moskva_selenium.html", "w", encoding="utf-8") as file:
        file.write(html_content)

    print("HTML-код сохранен")

finally:
    driver.quit()

HTML-код сохранен


В загруженном html-файле было найдено всего 80 отелей. Для Москвы это очень мало, но в этом нет проблемы otello - дело в том, что сайт не может загрузить на страницу сразу все тысячи отелей сразу, иначе бы страница грузилась очень долго. Обычно, догрузка происходит во время того, как пользователь скроллит вниз или нажимает на кнопки по типу "Загрузить еще". Otello одновременно использует оба таких подхода - нам надо воссоздать данное поведение с помощью дополнительных инструментов selenium

In [2]:
import logging
logger = logging.getLogger()
fhandler = logging.FileHandler(filename='otello_selenium.log', mode='a')
formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
fhandler.setFormatter(formatter)
logger.addHandler(fhandler)
logger.setLevel(logging.INFO)

In [8]:
chrome_options = Options()
# chrome_options.add_argument("--headless")  
service = Service('chromedriver.exe')

driver = webdriver.Chrome(service=service, options=chrome_options)

try:
    logging.info("Открытие страницы https://otello.ru/hotels/moskva")
    driver.get("https://otello.ru/hotels/moskva") 
    wait = WebDriverWait(driver, 5)

    # Закрываем баннер
    try:
        close_banner_button = wait.until(EC.element_to_be_clickable((By.XPATH, "//button[@data-n='wat-welcome-popup-close-button']")))
        close_banner_button.click()
        logging.info("Баннер закрыт.")
    except Exception as e:
        logging.error(f"Ошибка при закрытии баннера: {e}")
    
    time.sleep(2)  
    
    # Активируем панель с отелями
    try:
        action = ActionChains(driver)
        action.move_by_offset(0, 75).click().perform()
        logging.info("Окно активировано")
    except Exception as e:
        logging.error(f"Ошибка при активации окна: {e}")
    
    # Первичный скроллинг
    logging.info("Начинаем первичный скроллинг страницы...")
    end_time = time.time() + 30
    try:
        while time.time() < end_time:
            driver.find_element(By.TAG_NAME, "body").send_keys(Keys.PAGE_DOWN)
            time.sleep(2)
        logging.info("Первичный скроллинг завершен.")
    except Exception as e:
        logging.error(f"Ошибка при скроллинге: {e}")
    
    # Нажимаем 'Показать все'
    try:
        show_all_button = wait.until(EC.element_to_be_clickable((By.XPATH, "//button[@data-n='wat-hub-show-all-button']")))
        show_all_button.click()
        logging.info("Кнопка 'Показать все' нажата.")
    except Exception as e:
        logging.error(f"Ошибка при нажатии кнопки 'Показать все': {e}")
    
    time.sleep(5)
    
    # Еще раз активируем экран
    try:
        action.move_by_offset(0, 75).click().perform()  
        logging.info("Окно активировано")
    except Exception as e:
        logging.error(f"Ошибка при повторной активации окна: {e}")
    
    end_time = time.time() + 300
    collected_links = set()

    logging.info("Начинаем сбор ссылок: ")
    
    try:
        while time.time() < end_time:
            driver.find_element(By.TAG_NAME, "body").send_keys(Keys.PAGE_DOWN)
            time.sleep(2)
            
            hotel_links = driver.find_elements(By.XPATH, "//a[starts-with(@href, '/hotel/') and @draggable='false']")
            
            for link in hotel_links:
                url = link.get_attribute("href")
                if url and url not in collected_links:
                    collected_links.add(url)
                    # print(url)
                    logging.info(f"Собрана ссылка: {url}")
    except Exception as e:
        logging.error(f"Ошибка при сборе ссылок: {e}")
    
    logging.info(f"Собрано {len(collected_links)} ссылок на отели.")
    
    # Сохраняем HTML-код
    try:
        html_content = driver.page_source
        with open("otello_moskva_selenium_full.html", "w", encoding="utf-8") as file:
            file.write(html_content)
        logging.info("HTML-код сохранен")
    except Exception as e:
        logging.error(f"Ошибка при сохранении HTML-кода: {e}")
    
finally:
    driver.quit()
    logging.info("Веб-драйвер закрыт")


За пять минут было собрано всего около 160 ссылок... 10 619 ссылок, которые мы ранее получили с помощью API, будут делаться как минимум за час! Еще надо учитывать, что мы только получили информацию по Москве. Для получения информации по всей России нам потребуется вручную искать города с большим числом отелей и повторно запускать скрипт   

В данном проекте мы использовали selenium как один из способов добычи данных. Но рабочим вариантом для сайта otello является извлечение информации по открытому API